# Facebook Post Insights 自動化數據抓取工具

從 Facebook Graph API 獲取粉絲專頁的貼文層級數據，並彙整至 Google Sheets。

**數據包含**：貼文資訊、互動統計 (讚/留言/分享)、洞察指標 (觸及/點擊/影片觀看等)

## 1. 套件安裝與匯入

In [1]:
# 安裝必要套件
!pip install requests pandas gspread google-auth google-auth-oauthlib google-auth-httplib2

# 匯入必要函式庫
import requests
import pandas as pd
import gspread
from google.oauth2 import service_account
import json
from datetime import datetime, timedelta
import time
import os
from typing import List, Dict, Any, Optional

## 2. 設定檔與憑證

In [2]:
# 掛載 Google Drive (用於存取憑證檔案)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Facebook API 設定
FACEBOOK_CONFIG = {
    'app_id': '1085898272974442',
    'page_id': '103640919705348',
    'access_token': 'EAAPbnmTSpmoBPsBfJHKn3AZAHpCZC2XvkyYvZAcKmZCQHPnPL44i8yevD1PAxSGjaRRl87RFZB79vTKPGKAFdbT35HfZAcZApp5j76f6hkIXCsO6Sgmi06H7mbkOELkn3gfqLU2UKSDaDMfs5oeNyfBMfVpmG4GSlb1WF9GJ3pluweVV0mb2Jp79bWfkcvYZAx4eKNJtZApYZD',
    'api_version': 'v23.0'
}

# Google Sheets 設定
GOOGLE_SHEETS_CONFIG = {
    'credentials_path': '/content/drive/MyDrive/Colab Notebooks/gemini-api-reports-3a9837dee55c.json',
    'spreadsheet_name': 'Faceboook Insights Metrics_Data Warehouse',
    'worksheet_name': 'raw_data'
}

# 貼文層級 Insights 指標
# 根據測試結果，以下為可用的貼文層級指標（已移除無效指標）

POST_INSIGHTS_METRICS = [
    # 貼文互動指標
    'post_clicks',                  # 貼文點擊次數

    # 貼文觸及指標
    'post_impressions',             # 貼文曝光次數
    'post_impressions_unique',      # 貼文觸及人數
    'post_impressions_organic',     # 自然觸及的曝光次數
    'post_impressions_paid',        # 付費觸及的曝光次數

    # 貼文心情反應指標
    'post_reactions_like_total',    # 讚的總數
    'post_reactions_love_total',    # 大心的總數
    'post_reactions_wow_total',     # 哇的總數
    'post_reactions_haha_total',    # 哈的總數
    'post_reactions_sorry_total',   # 嗚的總數
    'post_reactions_anger_total',   # 怒的總數

    # 影片相關指標（僅適用於影片貼文）
    'post_video_views',             # 影片觀看次數
    'post_video_views_organic',     # 自然影片觀看次數
    'post_video_views_paid',        # 付費影片觀看次數
]

print("設定載入完成")
print(f"目標 Page ID: {FACEBOOK_CONFIG['page_id']}")
print(f"API 版本: {FACEBOOK_CONFIG['api_version']}")
print(f"貼文層級指標數量: {len(POST_INSIGHTS_METRICS)} 項")
print("\n已移除無效指標:")
print("  - post_engaged_users (API 不支援)")
print("  - post_clicks_unique (API 不支援)")
print("  - post_video_view_time (無數據返回)")

設定載入完成
目標 Page ID: 103640919705348
API 版本: v23.0
貼文層級指標數量: 14 項

已移除無效指標:
  - post_engaged_users (API 不支援)
  - post_clicks_unique (API 不支援)
  - post_video_view_time (無數據返回)


## 3. 核心功能函式

In [4]:
def test_facebook_api_connection(config: Dict[str, str]) -> bool:
    """
    測試 Facebook API 連接是否正常
    """
    try:
        url = f"https://graph.facebook.com/{config['api_version']}/{config['page_id']}"
        params = {
            'access_token': config['access_token'],
            'fields': 'id,name,fan_count'
        }

        response = requests.get(url, params=params)
        response.raise_for_status()

        data = response.json()
        print(f"✓ API 連接成功")
        print(f"  Page ID: {data.get('id')}")
        print(f"  Page Name: {data.get('name')}")
        print(f"  粉絲數: {data.get('fan_count', 'N/A')}")
        return True

    except requests.exceptions.RequestException as e:
        print(f"✗ API 連接失敗: {e}")
        return False
    except Exception as e:
        print(f"✗ 未預期的錯誤: {e}")
        return False


def fetch_page_posts(config: Dict[str, str], since: str, until: str, limit: int = 100) -> Optional[List[Dict]]:
    """
    從 Facebook API 獲取頁面貼文列表

    Args:
        config: Facebook API 設定
        since: 開始日期 (YYYY-MM-DD)
        until: 結束日期 (YYYY-MM-DD)
        limit: 每次請求的最大貼文數

    Returns:
        貼文列表，每個貼文包含基本資訊和統計數據
    """
    try:
        url = f"https://graph.facebook.com/{config['api_version']}/{config['page_id']}/posts"

        # 轉換日期為 Unix timestamp
        since_ts = int(datetime.strptime(since, '%Y-%m-%d').timestamp())
        until_ts = int(datetime.strptime(until, '%Y-%m-%d').timestamp()) + 86400  # 加一天以包含結束日期

        params = {
            'access_token': config['access_token'],
            'fields': 'id,message,created_time,permalink_url',
            'since': since_ts,
            'until': until_ts,
            'limit': limit
        }

        all_posts = []

        print(f"正在獲取貼文列表...")
        print(f"日期範圍: {since} 到 {until}")

        while url:
            response = requests.get(url, params=params)
            response.raise_for_status()

            data = response.json()
            posts = data.get('data', [])

            # 為每則貼文獲取額外資訊（反應、留言、分享數）
            for post in posts:
                post_id = post['id']

                # 獲取反應總數
                reactions_url = f"https://graph.facebook.com/{config['api_version']}/{post_id}/reactions"
                reactions_params = {'access_token': config['access_token'], 'summary': 'total_count', 'limit': 0}
                reactions_response = requests.get(reactions_url, params=reactions_params)
                if reactions_response.status_code == 200:
                    reactions_data = reactions_response.json()
                    post['reactions'] = {'summary': {'total_count': reactions_data.get('summary', {}).get('total_count', 0)}}

                # 獲取留言總數
                comments_url = f"https://graph.facebook.com/{config['api_version']}/{post_id}/comments"
                comments_params = {'access_token': config['access_token'], 'summary': 'total_count', 'limit': 0}
                comments_response = requests.get(comments_url, params=comments_params)
                if comments_response.status_code == 200:
                    comments_data = comments_response.json()
                    post['comments'] = {'summary': {'total_count': comments_data.get('summary', {}).get('total_count', 0)}}

                # 獲取分享數
                shares_url = f"https://graph.facebook.com/{config['api_version']}/{post_id}"
                shares_params = {'access_token': config['access_token'], 'fields': 'shares'}
                shares_response = requests.get(shares_url, params=shares_params)
                if shares_response.status_code == 200:
                    shares_data = shares_response.json()
                    post['shares'] = shares_data.get('shares', {})

            all_posts.extend(posts)

            # 檢查是否有下一頁
            paging = data.get('paging', {})
            url = paging.get('next')
            params = {}  # 下一頁的 URL 已包含所有參數

            print(f"  已獲取 {len(all_posts)} 則貼文...")

        print(f"✓ 共獲取 {len(all_posts)} 則貼文")
        return all_posts

    except requests.exceptions.RequestException as e:
        print(f"✗ API 請求失敗: {e}")
        if hasattr(e, 'response') and e.response is not None:
            try:
                error_detail = e.response.json()
                print(f"錯誤詳情: {error_detail}")
            except:
                pass
        return None
    except Exception as e:
        print(f"✗ 未預期的錯誤: {e}")
        return None


def fetch_post_insights(config: Dict[str, str], post_id: str, metrics: List[str]) -> Optional[Dict]:
    """
    從 Facebook API 獲取單一貼文的洞察數據

    Args:
        config: Facebook API 設定
        post_id: 貼文 ID
        metrics: 要獲取的指標列表

    Returns:
        貼文洞察數據字典
    """
    try:
        url = f"https://graph.facebook.com/{config['api_version']}/{post_id}/insights"
        params = {
            'access_token': config['access_token'],
            'metric': ','.join(metrics)
        }

        response = requests.get(url, params=params)
        response.raise_for_status()

        data = response.json()
        insights_data = data.get('data', [])

        # 將 insights 數據轉換為字典格式
        insights_dict = {}
        for metric_data in insights_data:
            metric_name = metric_data.get('name')
            values = metric_data.get('values', [])
            if values:
                # 取最新的值
                value = values[0].get('value', 0)
                insights_dict[metric_name] = value

        return insights_dict

    except requests.exceptions.RequestException as e:
        # 某些貼文可能無法獲取 insights（如分享的貼文）
        return {}
    except Exception as e:
        print(f"獲取貼文 {post_id} 的 insights 時發生錯誤: {e}")
        return {}

In [5]:
def process_posts_data(posts: List[Dict], page_id: str, page_name: str, fetch_date: str) -> pd.DataFrame:
    """
    處理貼文數據，轉換為結構化的 DataFrame

    Args:
        posts: 貼文列表（包含基本資訊和 insights）
        page_id: 粉絲專頁 ID
        page_name: 粉絲專頁名稱
        fetch_date: 數據抓取日期

    Returns:
        結構化的 DataFrame
    """
    processed_records = []

    for post in posts:
        record = {
            'fetch_date': fetch_date,
            'post_id': post.get('id', ''),
            'page_id': page_id,
            'page_name': page_name,
            'message': post.get('message', '')[:500],  # 限制長度避免過長
            'created_time': post.get('created_time', ''),
            'permalink_url': post.get('permalink_url', ''),
        }

        # 處理反應數（reactions）
        reactions_data = post.get('reactions', {})
        if isinstance(reactions_data, dict):
            record['reactions_count'] = reactions_data.get('summary', {}).get('total_count', 0)
        else:
            record['reactions_count'] = 0

        # 處理留言數
        comments_data = post.get('comments', {})
        if isinstance(comments_data, dict):
            record['comments_count'] = comments_data.get('summary', {}).get('total_count', 0)
        else:
            record['comments_count'] = 0

        # 處理分享數
        shares_data = post.get('shares', {})
        if isinstance(shares_data, dict):
            record['shares_count'] = shares_data.get('count', 0)
        else:
            record['shares_count'] = 0

        # 處理 insights 數據（如果有）
        insights = post.get('insights', {})
        for metric_name, value in insights.items():
            record[metric_name] = value

        processed_records.append(record)

    return pd.DataFrame(processed_records)


def validate_posts_dataframe(df: pd.DataFrame) -> bool:
    """
    驗證貼文 DataFrame 的結構是否正確
    """
    required_columns = ['fetch_date', 'post_id', 'page_id', 'page_name', 'created_time']

    if df.empty:
        print("⚠️ DataFrame 為空")
        return False

    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"⚠️ 缺少必要欄位: {missing_columns}")
        return False

    print(f"✓ DataFrame 驗證通過 ({len(df)} 則貼文)")
    return True

In [6]:
def setup_google_sheets_client(credentials_path: str) -> Optional[gspread.Client]:
    """
    設定 Google Sheets 客戶端
    """
    try:
        if not os.path.exists(credentials_path):
            print(f"✗ 找不到憑證檔案: {credentials_path}")
            return None

        # 設定 Google Sheets API 的權限範圍
        scope = [
            'https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive'
        ]

        # 載入服務帳戶憑證
        credentials = service_account.Credentials.from_service_account_file(
            credentials_path, scopes=scope)

        # 建立 gspread 客戶端
        client = gspread.authorize(credentials)

        print("✓ Google Sheets 客戶端設定成功")
        return client

    except Exception as e:
        print(f"✗ Google Sheets 客戶端設定失敗: {e}")
        return None


def write_to_google_sheets(client: gspread.Client, config: Dict[str, str], df: pd.DataFrame) -> bool:
    """
    將 DataFrame 寫入 Google Sheets，確保欄位順序與名稱正確，並避免重複記錄
    """
    try:
        # 開啟試算表
        spreadsheet = client.open(config['spreadsheet_name'])
        worksheet = spreadsheet.worksheet(config['worksheet_name'])

        # 定義正確的欄位順序
        expected_columns = [
            'fetch_date',
            'post_id',
            'page_id',
            'page_name',
            'message',
            'created_time',
            'permalink_url',
            'reactions_count',
            'comments_count',
            'shares_count',
            'post_clicks',
            'post_impressions',
            'post_impressions_unique',
            'post_impressions_organic',
            'post_impressions_paid',
            'post_reactions_like_total',
            'post_reactions_love_total',
            'post_reactions_wow_total',
            'post_reactions_haha_total',
            'post_reactions_sorry_total',
            'post_reactions_anger_total',
            'post_video_views',
            'post_video_views_organic',
            'post_video_views_paid'
        ]

        # 確保 DataFrame 包含所有必要欄位（缺少的填 0 或空字串）
        for col in expected_columns:
            if col not in df.columns:
                if col in ['fetch_date', 'post_id', 'page_id', 'page_name', 'message', 'created_time', 'permalink_url']:
                    df[col] = ''
                else:
                    df[col] = 0

        # 按照正確順序重新排列欄位
        df = df[expected_columns]

        # 檢查工作表是否為空或標題列不正確
        existing_data = worksheet.get_all_values()

        if not existing_data or not existing_data[0]:
            # 工作表為空，寫入標題列
            worksheet.append_row(expected_columns)
            print("✓ 已寫入標題列")
            existing_data = [expected_columns]
        else:
            # 檢查現有標題列是否正確
            existing_headers = existing_data[0]
            if existing_headers != expected_columns:
                print(f"⚠️  警告：工作表標題列與預期不符")
                print(f"   現有標題: {existing_headers[:5]}... ({len(existing_headers)} 個欄位)")
                print(f"   預期標題: {expected_columns[:5]}... ({len(expected_columns)} 個欄位)")
                print("   自動更新標題列...")
                worksheet.delete_rows(1)
                worksheet.insert_row(expected_columns, 1)
                print("✓ 已更新標題列")
                # 重新獲取數據
                existing_data = worksheet.get_all_values()

        # 檢查重複記錄
        # 建立現有記錄的 post_id + fetch_date 組合集合
        existing_records = set()
        if len(existing_data) > 1:  # 有數據行（不只標題）
            # 找到 post_id 和 fetch_date 的索引
            headers = existing_data[0]
            try:
                post_id_idx = headers.index('post_id')
                fetch_date_idx = headers.index('fetch_date')

                for row in existing_data[1:]:  # 跳過標題行
                    if len(row) > max(post_id_idx, fetch_date_idx):
                        key = f"{row[post_id_idx]}_{row[fetch_date_idx]}"
                        existing_records.add(key)

                print(f"✓ 已載入 {len(existing_records)} 筆現有記錄")
            except ValueError as e:
                print(f"⚠️  無法找到必要欄位索引: {e}")

        # 過濾出新記錄
        new_rows = []
        duplicate_count = 0

        for idx, row in df.iterrows():
            key = f"{row['post_id']}_{row['fetch_date']}"
            if key not in existing_records:
                new_rows.append(row.tolist())
            else:
                duplicate_count += 1

        print(f"\n數據去重結果:")
        print(f"  總記錄數: {len(df)}")
        print(f"  新記錄: {len(new_rows)}")
        print(f"  重複記錄（已跳過）: {duplicate_count}")

        # 批次寫入新數據
        if new_rows:
            worksheet.append_rows(new_rows)
            print(f"\n✓ 已寫入 {len(new_rows)} 筆新數據到 Google Sheets")
            print(f"   欄位數: {len(expected_columns)}")
            return True
        else:
            print("\n⚠️  無新數據需要寫入（所有記錄已存在）")
            return True  # 視為成功，因為沒有錯誤

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"✗ 找不到試算表: {config['spreadsheet_name']}")
        return False
    except gspread.exceptions.WorksheetNotFound:
        print(f"✗ 找不到工作表: {config['worksheet_name']}")
        return False
    except Exception as e:
        print(f"✗ 寫入 Google Sheets 失敗: {e}")
        import traceback
        traceback.print_exc()
        return False

## 4. 系統測試與指標驗證

In [7]:
# 測試 Facebook API 連接
print("=== Facebook API 連接測試 ===")
api_connected = test_facebook_api_connection(FACEBOOK_CONFIG)

if not api_connected:
    print("\n⚠️ API 連接失敗，請檢查 Access Token 是否有效")
    print("建議檢查事項:")
    print("1. Access Token 是否已過期")
    print("2. App 是否有適當的權限")
    print("3. Page ID 是否正確")

# 測試 Google Sheets 連接
print("\n=== Google Sheets 連接測試 ===")
sheets_client = setup_google_sheets_client(GOOGLE_SHEETS_CONFIG['credentials_path'])

if not sheets_client:
    print("\n⚠️ Google Sheets 連接失敗")
    print("建議檢查事項:")
    print("1. 憑證檔案路徑是否正確")
    print("2. 服務帳戶是否有試算表的編輯權限")
    print("3. Drive 是否已掛載")

=== Facebook API 連接測試 ===
✓ API 連接成功
  Page ID: 103640919705348
  Page Name: 綠色公民行動聯盟Green Citizens' Action Alliance
  粉絲數: 123006

=== Google Sheets 連接測試 ===
✓ Google Sheets 客戶端設定成功


In [8]:
# 測試貼文獲取功能
print("\n=== 貼文獲取測試 ===")

if api_connected:
    # 測試獲取最近 3 天的貼文
    test_until = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    test_since = (datetime.now() - timedelta(days=3)).strftime('%Y-%m-%d')

    print(f"測試日期範圍: {test_since} 到 {test_until}")

    test_posts = fetch_page_posts(FACEBOOK_CONFIG, test_since, test_until)

    if test_posts:
        print(f"\n✓ 成功獲取 {len(test_posts)} 則貼文")

        if test_posts:
            print("\n第一則貼文資訊:")
            first_post = test_posts[0]
            print(f"  ID: {first_post.get('id')}")
            print(f"  發布時間: {first_post.get('created_time')}")
            print(f"  訊息: {first_post.get('message', '')[:100]}...")

            reactions = first_post.get('reactions', {})
            if isinstance(reactions, dict):
                print(f"  反應數: {reactions.get('summary', {}).get('total_count', 0)}")

            comments = first_post.get('comments', {})
            if isinstance(comments, dict):
                print(f"  留言數: {comments.get('summary', {}).get('total_count', 0)}")

            shares = first_post.get('shares', {})
            if isinstance(shares, dict):
                print(f"  分享數: {shares.get('count', 0)}")

        # 測試獲取 insights
        if test_posts:
            print(f"\n測試獲取第一則貼文的 Insights...")
            post_id = test_posts[0].get('id')
            insights = fetch_post_insights(FACEBOOK_CONFIG, post_id, POST_INSIGHTS_METRICS)

            if insights:
                print(f"✓ 成功獲取 {len(insights)} 項 insights 指標")
                print(f"\n可用的 Insights 指標:")
                for metric_name, value in list(insights.items())[:5]:
                    print(f"  - {metric_name}: {value}")
                if len(insights) > 5:
                    print(f"  ... 及其他 {len(insights) - 5} 項指標")
            else:
                print("⚠️ 該貼文無法獲取 insights 數據")
    else:
        print("⚠️ 該日期範圍內無貼文或獲取失敗")
else:
    print("跳過貼文獲取測試 (API 連接失敗)")


=== 貼文獲取測試 ===
測試日期範圍: 2025-10-04 到 2025-10-06
正在獲取貼文列表...
日期範圍: 2025-10-04 到 2025-10-06
  已獲取 0 則貼文...
✓ 共獲取 0 則貼文
⚠️ 該日期範圍內無貼文或獲取失敗


## 5. 主要執行程序

In [9]:
def main_posts_collection(since_date: str = None, until_date: str = None, include_insights: bool = True) -> bool:
    """
    主要的貼文數據蒐集流程

    Args:
        since_date: 開始日期 (預設為90天前)
        until_date: 結束日期 (預設為昨天)
        include_insights: 是否包含 insights 指標

    Returns:
        是否成功
    """
    # 設定日期範圍 - 預設為最近 90 天
    if until_date is None:
        until_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    if since_date is None:
        since_date = (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d')

    fetch_date = datetime.now().strftime('%Y-%m-%d')

    print(f"=== 開始貼文數據蒐集 ===")
    print(f"日期範圍: {since_date} 到 {until_date}")
    print(f"執行日期: {fetch_date}")
    print(f"包含 Insights: {include_insights}")

    # 檢查前置條件
    if not api_connected:
        print("✗ Facebook API 未連接")
        return False

    if not sheets_client:
        print("✗ Google Sheets 未連接")
        return False

    # 獲取頁面資訊
    try:
        url = f"https://graph.facebook.com/{FACEBOOK_CONFIG['api_version']}/{FACEBOOK_CONFIG['page_id']}"
        params = {'access_token': FACEBOOK_CONFIG['access_token'], 'fields': 'id,name'}
        response = requests.get(url, params=params)
        response.raise_for_status()
        page_data = response.json()
        page_name = page_data.get('name', '')
        print(f"\n粉絲專頁: {page_name}")
    except Exception as e:
        print(f"✗ 無法獲取頁面資訊: {e}")
        return False

    # 從 Facebook API 獲取貼文列表
    print(f"\n步驟 1: 獲取貼文列表")
    posts = fetch_page_posts(FACEBOOK_CONFIG, since_date, until_date)

    if posts is None:
        print("✗ 無法獲取貼文")
        return False

    if not posts:
        print("⚠️ 該日期範圍內無貼文")
        return False

    print(f"✓ 成功獲取 {len(posts)} 則貼文")

    # 獲取每則貼文的 insights 數據
    if include_insights:
        print(f"\n步驟 2: 獲取貼文 Insights 數據")
        success_count = 0
        skip_count = 0

        for i, post in enumerate(posts, 1):
            post_id = post.get('id')
            print(f"  處理 {i}/{len(posts)}: {post_id}", end='')

            insights = fetch_post_insights(FACEBOOK_CONFIG, post_id, POST_INSIGHTS_METRICS)

            if insights:
                post['insights'] = insights
                success_count += 1
                print(f" - ✓ ({len(insights)} 項指標)")
            else:
                post['insights'] = {}
                skip_count += 1
                print(f" - ⊘ (無 insights)")

            # 避免請求過於頻繁
            time.sleep(0.2)

        print(f"\n✓ Insights 數據獲取完成")
        print(f"  成功: {success_count} 則")
        print(f"  跳過: {skip_count} 則")

    # 處理數據
    print(f"\n步驟 3: 處理數據")
    df = process_posts_data(posts, FACEBOOK_CONFIG['page_id'], page_name, fetch_date)

    if not validate_posts_dataframe(df):
        print("✗ 數據驗證失敗")
        return False

    # 顯示數據預覽
    print(f"\n=== 數據預覽 ===")
    print(f"欄位: {list(df.columns)}")
    print(f"\n前 3 則貼文:")
    # 只顯示存在的欄位
    display_cols = ['post_id', 'created_time', 'reactions_count', 'comments_count', 'shares_count']
    print(df[display_cols].head(3))

    print(f"\n數據摘要:")
    print(f"- 總貼文數: {len(df)}")
    print(f"- 日期範圍: {df['created_time'].min()} 到 {df['created_time'].max()}")
    print(f"- 總互動數: 反應 {df['reactions_count'].sum()}, 留言 {df['comments_count'].sum()}, 分享 {df['shares_count'].sum()}")

    # 寫入 Google Sheets
    print(f"\n步驟 4: 寫入 Google Sheets")
    success = write_to_google_sheets(sheets_client, GOOGLE_SHEETS_CONFIG, df)

    if success:
        print(f"\n🎉 貼文數據蒐集完成!")
        print(f"已將數據寫入 Google Sheets")
        return True
    else:
        print(f"\n❌ 貼文數據蒐集失敗")
        return False


# 執行主程序
if 'api_connected' in locals() and 'sheets_client' in locals():
    # 預設收集最近 90 天的貼文
    success = main_posts_collection()
else:
    print("請先執行上述所有測試單元格")

=== 開始貼文數據蒐集 ===
日期範圍: 2025-07-09 到 2025-10-06
執行日期: 2025-10-07
包含 Insights: True

粉絲專頁: 綠色公民行動聯盟Green Citizens' Action Alliance

步驟 1: 獲取貼文列表
正在獲取貼文列表...
日期範圍: 2025-07-09 到 2025-10-06
  已獲取 67 則貼文...
✓ 共獲取 67 則貼文
✓ 成功獲取 67 則貼文

步驟 2: 獲取貼文 Insights 數據
  處理 1/67: 103640919705348_1118586260465470 - ✓ (14 項指標)
  處理 2/67: 103640919705348_1118413803816049 - ✓ (14 項指標)
  處理 3/67: 103640919705348_1117610163896413 - ✓ (14 項指標)
  處理 4/67: 103640919705348_1117417193915710 - ✓ (14 項指標)
  處理 5/67: 103640919705348_1116756210648475 - ✓ (14 項指標)
  處理 6/67: 103640919705348_780669138147055 - ✓ (14 項指標)
  處理 7/67: 103640919705348_1114945644162865 - ✓ (14 項指標)
  處理 8/67: 103640919705348_1112455191078577 - ✓ (14 項指標)
  處理 9/67: 103640919705348_1112292774428152 - ✓ (14 項指標)
  處理 10/67: 103640919705348_2263017100887631 - ✓ (14 項指標)
  處理 11/67: 103640919705348_1105779671746129 - ✓ (14 項指標)
  處理 12/67: 103640919705348_1104896365167793 - ✓ (14 項指標)
  處理 13/67: 103640919705348_1448156213119081 - ✓ (14 項指標)
  處理